# Recursive Ekphrasis Gym

## Processing `(possessor,possessed)` pairs

 

In [1]:
! wget https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/possessor2possessed_tuples_with_filenumber.json

--2022-05-03 01:58:47--  https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/possessor2possessed_tuples_with_filenumber.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18738032 (18M) [text/plain]
Saving to: ‘possessor2possessed_tuples_with_filenumber.json.2’

possessor2possessed 100%[===================>]  17.87M   112MB/s    in 0.2s    

2022-05-03 01:58:47 (112 MB/s) - ‘possessor2possessed_tuples_with_filenumber.json.2’ saved [18738032/18738032]



In [2]:
import json

with open('possessor2possessed_tuples_with_filenumber.json','r') as f:
  possessor2possessed_tuples_with_filenumber_lists = json.load(f)

In [3]:
possessor2possessed_tuples_with_filenumber_lists[:4]

[['2068', ['captain', 'property']],
 ['2068', ['dealer', 'wagon']],
 ['2068', ['takin', 'carpets']],
 ['2068', ['body', 'eyesight']]]

In [4]:
from collections import defaultdict

possessor2possessed = defaultdict(list)

In [5]:
for i in possessor2possessed_tuples_with_filenumber_lists:
  filenumber = i[0]
  possessor = i[1][0]
  possessed = i[1][1] 
  possessor2possessed[possessor].append((possessed,filenumber))

In [6]:
possessor2possessed["curate"][:10]

[('garden', '43754'),
 ('daughters', '48198'),
 ('side', '28684'),
 ('children', '145'),
 ('children', '145'),
 ('son', '145'),
 ('place', '22008'),
 ('love', '2686'),
 ('wife', '11825'),
 ('ministry', '46570')]

Using WordNet to make sure a token is a word.

In [7]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
wn.synsets("hat",pos=wn.NOUN)

[Synset('hat.n.01'), Synset('hat.n.02')]

In [9]:
for key,values in possessor2possessed.items():
  values_unique_per_file = [thing for thing,f in list(set(values))]
  values_unique_per_file = [v for v in values_unique_per_file if wn.synsets(v,pos=wn.NOUN)!=[]] ## spellcheck via wordnet
  possessor2possessed[key] = values_unique_per_file

In [10]:
possessor2possessed["curate"][:5]

['egg', 'life', 'reach', 'turn', 'place']

In [11]:
import itertools
all_possessed = list(itertools.chain(*possessor2possessed.values()))

In [12]:
len(all_possessed)

521389

In [13]:
all_possessed[:10]

['sails',
 'mood',
 'part',
 'heart',
 'oilskins',
 'stateroom',
 'shoulder',
 'office',
 'head',
 'deportment']

In [14]:
from collections import defaultdict

possessed2count = defaultdict(int)

for w in all_possessed:
  possessed2count[w]+=1

In [15]:
(possessed2count['crest']-10)/len(all_possessed)

0.00014960039433129584

In [16]:
def specificity_test(word,targetlist,allwordlist=all_possessed,timesgreater=30):
  ratio_in_target = targetlist.count(word)/len(targetlist)
  ratio_general = (possessed2count[word]-targetlist.count(word))/len(all_possessed)
  if (ratio_in_target/timesgreater) > ratio_general:
    return True
  else:
    return False
  #return (ratio_in_target,ratio_general)

[(w,specificity_test(w,w,possessor2possessed['curate'])) for w in possessor2possessed['curate']]


[('egg', True),
 ('life', False),
 ('reach', True),
 ('turn', True),
 ('place', True),
 ('increase', True),
 ('mouth', True),
 ('wife', True),
 ('calves', True),
 ('case', True),
 ('budget', True),
 ('house', False),
 ('parrot', True),
 ('lecture', True),
 ('friends', True),
 ('head', False),
 ('visit', True),
 ('rooms', True),
 ('daughter', False),
 ('hand', False),
 ('voice', False),
 ('tone', True),
 ('remarks', True),
 ('chamber', True),
 ('daughter', False),
 ('mention', True),
 ('house', False),
 ('unselfishness', True),
 ('daughter', False),
 ('openness', True),
 ('consent', True),
 ('home', True),
 ('escort', True),
 ('cape', True),
 ('master', True),
 ('daughter', False),
 ('house', False),
 ('thoughts', True),
 ('tuition', True),
 ('wife', True),
 ('heart', False),
 ('family', True),
 ('ad', True),
 ('neighbourhood', True),
 ('knavery', True),
 ('lodging', True),
 ('hat', True),
 ('face', False),
 ('remark', True),
 ('stipend', True),
 ('face', False),
 ('wife', True),
 ('per

In [17]:
possessor2possessed_filtered = dict()

In [18]:
from tqdm import tqdm

In [19]:
for key,values in tqdm(possessor2possessed.items()):
  specific_to_possessor = []
  filtered_out = []
  for possessed in values:
    # if possessed in specific_to_possessor:
    #   specific_to_possessor.append(possessed)
    # elif possessed in filtered_out:
    #   pass
    # else:
    if specificity_test(possessed,values)==True:
      specific_to_possessor.append(possessed)
  possessor2possessed_filtered[key] = specific_to_possessor

100%|██████████| 16254/16254 [05:05<00:00, 53.24it/s]   


In [20]:
possessor2possessed_filtered['cherry']

['leaves', 'voice', 'time']

In [21]:
with open('possessor2possessed.json','w') as f:
  json.dump(possessor2possessed,f)

***